In [1]:
import os
import re

import pandas as pd

from dataclasses import dataclass
from tabulate import tabulate
from huggingface_hub import (
    login,
    HfApi,
    list_models,
    hf_hub_download,
    ModelCard,
    ModelCardData,
    update_repo_visibility
)

In [3]:
# Hub Login
hf_token = os.environ.get("HF_TOKEN")

login(token=hf_token, add_to_git_credential=False)
api = HfApi()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/stefan/.cache/huggingface/token
Login successful


In [4]:
model_search_string = "stefan-it/autotrain-flair-georgian-ner-xlm_r_large-"
trained_models = list(list_models(author="stefan-it", search=model_search_string))

In [5]:
# Display model ids

model_ids = [model.modelId for model in trained_models]
print("\n".join(model_ids))

stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-1
stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-2
stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-3
stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-4
stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-5


In [7]:
hyper_parameter_regex = re.compile(".*(?P<batch_size>bs.*?)-(?P<epoch>e.*?)-(?P<learning_rate>lr.*?)-(?P<seed>\d+)$")

@dataclass
class ModelInfo:
    model_id: str
    dev_f1_score: float
    batch_size: int
    epoch: int
    learning_rate: float
    seed: int

In [8]:
# Get best model score from training log
def get_best_model_score(input_file: str) -> float:
    all_dev_results = []
    with open(input_file, "rt") as f_p:
        for line in f_p:
            line = line.rstrip()
            if "f1-score (micro avg)" in line:
                dev_result = line.split(" ")[-1]
                all_dev_results.append(dev_result)

        return max([float(value) for value in all_dev_results])

In [10]:
# Get best f1-score from Flair training log for each model
# and construct nice ModelInfo instance
model_infos = []

hyper_params = ["batch_size", "epoch", "learning_rate", "seed"]

for model_id in model_ids:
    # Fetch Flair training log
    training_log_file = hf_hub_download(repo_id=model_id, filename="training.log")
    dev_f1_score = get_best_model_score(training_log_file)
    
    matches = hyper_parameter_regex.match(model_id)
    
    params = [matches.group(hyper_param) for hyper_param in hyper_params]
    model_info_args = (model_id, dev_f1_score, *params)

    model_infos.append(ModelInfo(*model_info_args))

In [14]:
print(model_infos)

[ModelInfo(model_id='stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-1', dev_f1_score=0.9005, batch_size='bs4', epoch='e10', learning_rate='lr5e-06', seed='1'), ModelInfo(model_id='stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-2', dev_f1_score=0.9012, batch_size='bs4', epoch='e10', learning_rate='lr5e-06', seed='2'), ModelInfo(model_id='stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-3', dev_f1_score=0.9069, batch_size='bs4', epoch='e10', learning_rate='lr5e-06', seed='3'), ModelInfo(model_id='stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-4', dev_f1_score=0.905, batch_size='bs4', epoch='e10', learning_rate='lr5e-06', seed='4'), ModelInfo(model_id='stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-5', dev_f1_score=0.9048, batch_size='bs4', epoch='e10', learning_rate='lr5e-06', seed='5')]


In [15]:
# Find out all hyper-parameter configurations: (Batch Size, Epoch, Learning Rate)

configurations = set([
    f"{model_info.batch_size}-{model_info.epoch}-{model_info.learning_rate}" for model_info in model_infos
])

print(configurations)

{'bs4-e10-lr5e-06'}


In [16]:
df_data = [
    [configuration, model_info.dev_f1_score
    ]
    for model_info in model_infos
    for configuration in configurations
    
    if f"{model_info.batch_size}-{model_info.epoch}-{model_info.learning_rate}" == configuration
]

In [21]:
df = pd.DataFrame(df_data, columns=["Configuration", "F1-Score"])
print(df)

     Configuration  F1-Score
0  bs4-e10-lr5e-06    0.9005
1  bs4-e10-lr5e-06    0.9012
2  bs4-e10-lr5e-06    0.9069
3  bs4-e10-lr5e-06    0.9050
4  bs4-e10-lr5e-06    0.9048


In [22]:
df = df[df.duplicated("Configuration", keep=False)].groupby("Configuration")["F1-Score"].apply(list).reset_index()

In [23]:
print(df)

     Configuration                                 F1-Score
0  bs4-e10-lr5e-06  [0.9005, 0.9012, 0.9069, 0.905, 0.9048]


In [24]:
# How many seeds do we have?
number_seeds = len(df["F1-Score"].to_list()[0])
seed_columns = [f"Seed {index + 1}" for index in range(number_seeds)]

split = pd.DataFrame(df["F1-Score"].to_list(), columns=seed_columns)

In [25]:
print(split)

   Seed 1  Seed 2  Seed 3  Seed 4  Seed 5
0  0.9005  0.9012  0.9069   0.905  0.9048


In [26]:
final_df = pd.concat([df, split], axis=1)
final_df = final_df.drop("F1-Score", axis=1)
final_df["Average"] = final_df[seed_columns].mean(axis=1).round(4)
final_df["Std."] = final_df[seed_columns].std(axis=1).round(4)

final_df.sort_values(by="Average", ascending=False, inplace=True)

print(final_df)

     Configuration  Seed 1  Seed 2  Seed 3  Seed 4  Seed 5  Average    Std.
0  bs4-e10-lr5e-06  0.9005  0.9012  0.9069   0.905  0.9048   0.9037  0.0027


In [27]:
print(final_df.to_markdown(index=False))

| Configuration   |   Seed 1 |   Seed 2 |   Seed 3 |   Seed 4 |   Seed 5 |   Average |   Std. |
|:----------------|---------:|---------:|---------:|---------:|---------:|----------:|-------:|
| bs4-e10-lr5e-06 |   0.9005 |   0.9012 |   0.9069 |    0.905 |   0.9048 |    0.9037 | 0.0027 |


In [28]:
# But we need results table with nice references to actual models on the hub
# Including highlighted F1-Score (bold) of current model
def get_results_table(final_df, model_infos, current_model_info):
    headers = final_df.columns.tolist()
    
    sorted_configurations = final_df["Configuration"].tolist()
    sorted_averages = final_df["Average"].tolist()
    sorted_stds = final_df["Std."].tolist()
    
    rows = []
    
    ref_counter = 1
    ref_list = []
    
    for sorted_configuration, sorted_average, sorted_std in zip(sorted_configurations, sorted_averages, sorted_stds):
        current_row = [f"`{sorted_configuration}`"]
        
        for model_info in model_infos:
            if f"{model_info.batch_size}-{model_info.epoch}-{model_info.learning_rate}" == sorted_configuration:
                # Model belongs to current configuration
                # Score + Model link looks like: [0.8585][4]
                seed_entry = f"[{model_info.dev_f1_score}][{ref_counter}]"
                
                # Special case: model is current model, so we need to bold it
                if model_info == current_model_info:
                    seed_entry = f"[**{model_info.dev_f1_score}**][{ref_counter}]"
    
                current_row.append(seed_entry)
    
                ref_list.append(f"[{ref_counter}]: https://hf.co/{model_info.model_id}")
                ref_counter += 1
    
        current_row.append(f"{sorted_average} ± {sorted_std}")
        
        rows.append(current_row)

    results_table = tabulate(rows, headers=headers, tablefmt="github") + "\n\n" + "\n".join(ref_list)
    
    return results_table
    
# Test it:
test_results_table = get_results_table(final_df, model_infos, model_infos[4])
print(model_infos[4])
print("\nTest Results Table:\n")
print(test_results_table)

ModelInfo(model_id='stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-5', dev_f1_score=0.9048, batch_size='bs4', epoch='e10', learning_rate='lr5e-06', seed='5')

Test Results Table:

| Configuration     | Seed 1      | Seed 2      | Seed 3      | Seed 4     | Seed 5          | Average         |
|-------------------|-------------|-------------|-------------|------------|-----------------|-----------------|
| `bs4-e10-lr5e-06` | [0.9005][1] | [0.9012][2] | [0.9069][3] | [0.905][4] | [**0.9048**][5] | 0.9037 ± 0.0027 |

[1]: https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-1
[2]: https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-2
[3]: https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-3
[4]: https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-4
[5]: https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-5


In [29]:
base_model = "xlm-roberta-large"
base_model_short = "XLM-R Large"

# Determine them automatically
batch_sizes = set([
    model_info.batch_size.replace("bs", "") for model_info in model_infos
])

learning_rates = set([
    model_info.learning_rate.replace("lr", "") for model_info in model_infos
])

widget_text = """
ამით თავისი ქადაგება დაასრულა და დაბრუნდა იერუსალიმში . ერთ-ერთ გარე კედელზე არსებობს ერნესტო ჩე გევარას პორტრეტი . შაკოსკა“ ინახება ბრაზილიაში , სან-პაულუს ხელოვნების მუზეუმში .
"""

In [30]:
# Now let's create model cards for all fine-tuned models 🤗
# Use this to create follow-up PRs when modifications are made in model card template!

commit_message = "readme: add initial version of model card"
commit_description = "Hey,\n\nthis PR adds the initial version of model card."
create_pr = True

for model in model_infos:
    current_results_table = get_results_table(final_df, model_infos, model)
    card_data = ModelCardData()
    card = ModelCard.from_template(card_data, template_path="model_card_template.md",
                                   base_model=base_model,
                                   base_model_short=base_model_short,
                                   batch_sizes=f'[{", ".join([f"`{bs}`" for bs in batch_sizes ])}]',
                                   learning_rates=f'[{", ".join([f"`{lr}`" for lr in learning_rates ])}]',
                                   results=current_results_table,
                                   widget_text=widget_text.strip()
                                  )

    commit_url = card.push_to_hub(repo_id=model.model_id,
                                  create_pr=create_pr,
                                  commit_message=commit_message,
                                  commit_description=commit_description)
    
    print(commit_url + "\n")

https://huggingface.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-1/blob/refs%2Fpr%2F1/README.md

https://huggingface.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-2/blob/refs%2Fpr%2F1/README.md

https://huggingface.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-3/blob/refs%2Fpr%2F1/README.md

https://huggingface.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-4/blob/refs%2Fpr%2F1/README.md

https://huggingface.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-5/blob/refs%2Fpr%2F1/README.md



In [31]:
# Now make repositories publicly visible
for model in model_infos:
    print(f"Update visibility to True for repo https://hf.co/{model.model_id}")
    update_repo_visibility(repo_id=model.model_id, private=False)

Update visibility to True for repo https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-1
Update visibility to True for repo https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-2
Update visibility to True for repo https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-3
Update visibility to True for repo https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-4
Update visibility to True for repo https://hf.co/stefan-it/autotrain-flair-georgian-ner-xlm_r_large-bs4-e10-lr5e-06-5
